In [ ]:
!pip install selenium
!pip install tqdm

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

from tqdm import tqdm
import pandas as pd
import re

import tickers

import time

In [2]:
def getListOfTicketNames(dict):
    list = []
    for key in dict.keys():
        list.append(key)
          
    return list

In [3]:
def getIncomeStatement(link):
    if (link == "https://www.investing.com/equities/linde-plc"):
        resultIncomeStatementLink = link + "-income-statement?cid=942017"
    else:
        resultIncomeStatementLink = link + "-income-statement"

    return resultIncomeStatementLink

In [4]:
def getBalanceSheet(link):
    if (link == "https://www.investing.com/equities/linde-plc"):
        resultBalanceSheetLink = link + "-balance-sheet?cid=942017"
    else:
        resultBalanceSheetLink = link + "-balance-sheet"

    return resultBalanceSheetLink

In [5]:
def getCashFlow(link):
    if (link == "https://www.investing.com/equities/linde-plc"):
        resultCashFlowLink = link + "-cash-flow?cid=942017"
    else:
        resultCashFlowLink = link + "-cash-flow"
    return resultCashFlowLink

In [6]:
res_list_income = {}
res_list_balance = {}
res_list_cash = {}

links = tickers.getTicketLinks()
tkts = getListOfTicketNames(links)

Потребуются Firefox и данный плагин для корректной работы скрипта

https://addons.mozilla.org/ru/firefox/addon/ublock-origin/?utm_source=addons.mozilla.org&utm_medium=referral&utm_content=search

In [8]:
driver = webdriver.Firefox()
extension_dir = '/Users/vladoo/Library/Application Support/Firefox/Profiles/dqobynzc.default-release/extensions/'
extensions = [
    'uBlock0@raymondhill.net.xpi',
    ]
for extension in extensions:
    driver.install_addon(extension_dir + extension, temporary=True)
        
print("driver ready")   

tick_num = 0
for i in tqdm(links):
    
    driver.get(getIncomeStatement(links[i]))
    time.sleep(2)
    try:
        driver.find_element_by_xpath('/html/body/div[5]/section/div[8]/div[1]/a[1]').click()
    except: 
        print(str(tkts[tick_num]) + 'XPATH not found. Trying another')
        driver.find_element_by_xpath('/html/body/div[5]/section/div[9]/div[1]/a[1]').click()
        
    time.sleep(5)
    res_list_income[tkts[tick_num]] = driver.find_elements_by_xpath('/html/body/div[5]/section/div[9]/table/tbody[2]/tr')

    n = 0
    for el_inc in res_list_income[tkts[tick_num]]:
        res_list_income[tkts[tick_num]][n] = el_inc.text
        n+=1
        
# ----------------------------------------------------------------------------------

    driver.get(getBalanceSheet(links[i]))
    time.sleep(2)
    try:
        driver.find_element_by_xpath('/html/body/div[5]/section/div[8]/div[1]/a[1]').click()
    except: 
        print(str(tkts[tick_num]) + 'XPATH not found. Trying another')
        driver.find_element_by_xpath('/html/body/div[5]/section/div[9]/div[1]/a[1]').click()
    
    time.sleep(5)
    res_list_balance[tkts[tick_num]] = driver.find_elements_by_xpath('/html/body/div[5]/section/div[9]/table/tbody[2]/tr')

    n = 0
    for el_bal in res_list_balance[tkts[tick_num]]:
        res_list_balance[tkts[tick_num]][n] = el_bal.text
        n+=1
            
# ----------------------------------------------------------------------------------

    driver.get(getCashFlow(links[i]))
    time.sleep(2)
    try:
        driver.find_element_by_xpath('/html/body/div[5]/section/div[8]/div[1]/a[1]').click()
    except: 
        print(str(tkts[tick_num]) + 'XPATH not found. Trying another')
        driver.find_element_by_xpath('/html/body/div[5]/section/div[9]/div[1]/a[1]').click()

    time.sleep(5)
    res_list_cash[tkts[tick_num]] = driver.find_elements_by_xpath('/html/body/div[5]/section/div[9]/table/tbody[2]/tr')

    n = 0
    for el_cash in res_list_cash[tkts[tick_num]]:
        res_list_cash[tkts[tick_num]][n] = el_cash.text
        n+=1
        
        
    tick_num += 1
    time.sleep(3)
    
    
driver.close()
print('close')

driver ready


  0%|          | 0/505 [00:00<?, ?it/s]<ipython-input-8-da4a7b8e43b5>:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('/html/body/div[5]/section/div[8]/div[1]/a[1]').click()
<ipython-input-8-da4a7b8e43b5>:23: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  res_list_income[tkts[tick_num]] = driver.find_elements_by_xpath('/html/body/div[5]/section/div[9]/table/tbody[2]/tr')
<ipython-input-8-da4a7b8e43b5>:35: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('/html/body/div[5]/section/div[8]/div[1]/a[1]').click()
<ipython-input-8-da4a7b8e43b5>:41: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  res_list_balance[tkts[tick_num]] = driver.find_elements_by_xpath('/html/body/div[5]/section/div[9]/table/tbody[2]/tr')
<ipython-input

close


res_list_income['MMM'] -- лист строк по тикеру <br>
res_list_income['MMM'][0] -- вывыести конкреткую строку по тикеру

In [9]:
patterns = [
re.compile("(?:^Revenue\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
re.compile("(?:^Cost of Revenue, Total\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
re.compile("(?:Total Assets\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
re.compile("(?:^Net Income\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
re.compile("(?:Accumulated Depreciation, Total\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
re.compile("(?:Capital Expenditures\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
re.compile("(?:Total Revenue\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
re.compile("(?:Total Current Assets\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
re.compile("(?:Total Current Liabilities\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
re.compile("(?:Cash and Short Term Investments\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
re.compile("(?:Notes Payable/Short Term Debt\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
re.compile("(?:Payable[/]Accrued\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
re.compile("(?:Other Liabilities, Total\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
re.compile("(?:Net Income Before Taxes\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
re.compile("(?:Changes in Working Capital\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
re.compile("(?:Retained Earnings [(]Accumulated Deficit[)]\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
re.compile("(?:Diluted Normalized EPS\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
re.compile("(?:Total Equity\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
re.compile("(?:Total Common Shares Outstanding\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)")
]

In [10]:
after_extract = {}

for ticker in tqdm(tkts):
    after_extract[ticker] = {}    
    for patt in range(len(patterns)):
        after_extract[ticker][patt+1] = ['-','-','-','-','-','-','-','-','-','-']

100%|██████████| 505/505 [00:00<00:00, 17023.98it/s]


In [11]:
for ticker in tqdm(tkts):    
    for patt in range(len(patterns)):
        for line_income in range(len(res_list_income[ticker])):
            if (re.search(patterns[patt], res_list_income[ticker][line_income])) is not None:
                temp_income = re.search(patterns[patt], res_list_income[ticker][line_income]).group().split()
                after_extract[ticker][patt+1] = temp_income
            
        for line_balance in range(len(res_list_balance[ticker])):
            if (re.search(patterns[patt], res_list_balance[ticker][line_balance])) is not None:
                temp_balance = re.search(patterns[patt], res_list_balance[ticker][line_balance]).group().split()
                after_extract[ticker][patt+1] = temp_balance
            
        for line_cash in range(len(res_list_cash[ticker])):
            if (re.search(patterns[patt], res_list_cash[ticker][line_cash])) is not None:
                temp_cash = re.search(patterns[patt], res_list_cash[ticker][line_cash]).group().split()
                after_extract[ticker][patt+1] = temp_cash

100%|██████████| 505/505 [00:01<00:00, 373.37it/s]


In [13]:
df_res = pd.DataFrame(columns=['Indicator', '2020','2019','2018','2017'])

indicators = ['revenue','cost of revenue','total assets','net income',
              'Accumulated Depreciation, Total','capital expenditures',
              'total revenue','total current assets','total current liabilities',
              'Cash and Short Term Investments','Notes Payable/Short Term Debt',
              'Payable/Accrued','Other Liabilities','Net Income Before Taxes',
              'Changes in Working Capital','retained earnings','Diluted Normalized EPS',
              'Total Equity','Total Common Shares Outstanding']

for t in tqdm(tkts):
    ind = [t,'','','','','','','','','','','','','','','','','','']
    d = {'Indicator': indicators,
         '2020':[after_extract[t][1][1],after_extract[t][2][4],after_extract[t][3][2],after_extract[t][4][2],after_extract[t][5][3],after_extract[t][6][2],after_extract[t][7][2],after_extract[t][8][3],after_extract[t][9][3],after_extract[t][10][5],after_extract[t][11][4],after_extract[t][12][1],after_extract[t][13][3],after_extract[t][14][4],after_extract[t][15][4],after_extract[t][16][4],after_extract[t][17][3],after_extract[t][18][2],after_extract[t][19][4]],
         '2019':[after_extract[t][1][2],after_extract[t][2][5],after_extract[t][3][3],after_extract[t][4][3],after_extract[t][5][4],after_extract[t][6][3],after_extract[t][7][3],after_extract[t][8][4],after_extract[t][9][4],after_extract[t][10][6],after_extract[t][11][5],after_extract[t][12][2],after_extract[t][13][4],after_extract[t][14][5],after_extract[t][15][5],after_extract[t][16][5],after_extract[t][17][4],after_extract[t][18][3],after_extract[t][19][5]], 
         '2018':[after_extract[t][1][3],after_extract[t][2][6],after_extract[t][3][4],after_extract[t][4][4],after_extract[t][5][5],after_extract[t][6][4],after_extract[t][7][4],after_extract[t][8][5],after_extract[t][9][5],after_extract[t][10][7],after_extract[t][11][6],after_extract[t][12][3],after_extract[t][13][5],after_extract[t][14][6],after_extract[t][15][6],after_extract[t][16][6],after_extract[t][17][5],after_extract[t][18][4],after_extract[t][19][6]], 
         '2017':[after_extract[t][1][4],after_extract[t][2][7],after_extract[t][3][5],after_extract[t][4][5],after_extract[t][5][6],after_extract[t][6][5],after_extract[t][7][5],after_extract[t][8][6],after_extract[t][9][6],after_extract[t][10][8],after_extract[t][11][7],after_extract[t][12][4],after_extract[t][13][6],after_extract[t][14][7],after_extract[t][15][7],after_extract[t][16][7],after_extract[t][17][6],after_extract[t][18][5],after_extract[t][19][7]]
        }
    df_temp = pd.DataFrame(d, index=ind) 
    df_res = df_res.append(df_temp)

100%|██████████| 505/505 [00:00<00:00, 559.71it/s]


In [14]:
df_res

,Indicator,2020,2019,2018,2017
MMM,revenue,32184,32136,32765,31657
,cost of revenue,16499,17160,16657,16062
,total assets,47344,44659,36500,37987
,net income,5384,4570,5349,4858
,"Accumulated Depreciation, Total",-17229,-16791,-16135,-16048
...,...,...,...,...,...
,Changes in Working Capital,-85,-151,112,-29
,retained earnings,5659,4427,3270,2109
,Diluted Normalized EPS,3.52,3.17,2.89,2.21
,Total Equity,3769,2708,2185,1770


In [16]:
df_res.to_excel('tables_usa_qf.xlsx')

In [ ]:
# # для добавления вручную, если некоторые тикеры отсутствуют после загрузки

# bad_ticker = ''

# driver = webdriver.Firefox()
# extension_dir = '/Users/vladoo/Library/Application Support/Firefox/Profiles/dqobynzc.default-release/extensions/'
# extensions = [
#     'uBlock0@raymondhill.net.xpi',
#     ]
# for extension in extensions:
#     driver.install_addon(extension_dir + extension, temporary=True)
        
# print("driver ready")  

# driver.get(getIncomeStatement(links[bad_ticker]))
# try:
#     driver.find_element_by_xpath('/html/body/div[5]/section/div[8]/div[1]/a[1]').click()
# except: 
#     driver.find_element_by_xpath('/html/body/div[5]/section/div[9]/div[1]/a[1]').click()
        
# time.sleep(3)
# res_list_income[bad_ticker] = driver.find_elements_by_xpath('/html/body/div[5]/section/div[9]/table/tbody[2]/tr')

# n = 0
# for el_inc in res_list_income[bad_ticker]:
#     res_list_income[bad_ticker][n] = el_inc.text
#     n+=1
    
# driver.close()